In [ ]:
# API client library
import googleapiclient.discovery
import pandas as pd
import sys
import os
sys.path.append('../..')
from src import getSecret
from googleapiclient.errors import HttpError

# API information
api_service_name = "youtube"
api_version = "v3"

# API key
DEVELOPER_KEY = getSecret('shanyas10_YOUTUBE_APIKEY')

# API client
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

# Query
request = youtube.search().list(part='id,snippet', q='food', type='video', maxResults=50)

try:
    response = request.execute()
except HttpError as error:
    print(f"An HTTP error {error.resp.status} occurred: {error.content}")


# Create a data frame from the response
# df = pd.json_normalize(response['items'])

# Print the data frame
# display(df)

In [ ]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.activities.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

# def main():
# Disable OAuthlib's HTTPS verification when running locally.
# *DO NOT* leave this option enabled in production.
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = '../../secrets/oauth2.0_youtubeAPI_secretKey.json'

# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_local_server(port=8090, open_browser=True)
# local

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

request = youtube.activities().list(
    part="id,snippet",
    maxResults=50,
    mine=True
)
response = request.execute()

print(response)

In [ ]:
request = youtube.playlists().list(
    part="snippet",
    mine=True,
    maxResults=50
)
response = request.execute()

for item in response['items']:
    print(item['snippet']['title'])

In [ ]:
request = youtube.channels().list(part='contentDetails')
response = request.execute()
# playlistId = response['items'][0]['contentDetails']['relatedPlaylists']['watchHistory']

In [ ]:
# put in a dataframe
import pandas as pd
watch_history = [item for item in response['items']]
df = pd.json_normalize(watch_history)
display(df)

In [ ]:

# Get the remaining quota
request = youtube.quota().get(part="quota")

# Execute the request
response = request.execute()

# Print the remaining quota
print(response['quota']['remaining'])

In [ ]:
import google.auth
from google.oauth2.credentials import Credentials
import googleapiclient.discovery
from googleapiclient.errors import HttpError

# Set up the API client
api_service_name = "youtube"
api_version = "v3"

# Authenticate the user
creds, project = google.auth.default(scopes=["https://www.googleapis.com/auth/youtube.readonly","https://www.googleapis.com/auth/youtube.force-ssl"])

# Build the API client
youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=creds)


request = youtube.activities().list(part="snippet,contentDetails")
# response = request.execute()

try:
    response = request.execute()
except HttpError as error:
    print(f"An HTTP error {error.resp.status} occurred: {error.content}")


# # Print the watch history
# for item in response["items"]:
#     print(item["snippet"]["title"])

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Replace with the path to your JSON credentials file
creds = service_account.Credentials.from_service_account_file('path/to/credentials.json')

# Create a YouTube API client
youtube = build('youtube', 'v3', credentials=creds)

# Define the parameters for the request
params = {
    'part': 'contentDetails',
    'mine': True,
    'maxResults': 50,
    'fields': 'items(contentDetails/videoId)',
    'activityType': 'watch'
}

# Send the request to the API
response = youtube.activities().list(**params).execute()

# Filter the list of activities to get only the watch history
watch_history = [item['contentDetails']['videoId'] for item in response['items']]

# Print the watch history
print(watch_history)


In [ ]:
import sys
import os
sys.path.append('../..')
print(os.getcwd())

# load from dict_recentlyPlayed.txt
import pandas as pd
import json
with open('../../data/dict_recentlyPlayed.txt', 'r') as f:
    dict_recentlyPlayed = json.load(f)

with open('../../data/dict_heartRate.txt', 'r') as f:
    dict_heartRate = json.load(f)

df_heartRate = pd.DataFrame.from_dict(dict_heartRate)
df_heartRate['datetime'] = pd.to_datetime(df_heartRate['date']) + pd.to_timedelta(df_heartRate['time'])
df_recentlyPlayed = pd.DataFrame.from_dict(dict_recentlyPlayed)
df_recentlyPlayed['datetime'] = pd.to_datetime(df_recentlyPlayed['played_at'])
# apply to EST
df_recentlyPlayed['datetime'] = df_recentlyPlayed['datetime'] - pd.Timedelta(hours=4)

In [ ]:
df_heartRate

In [ ]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import pandas as pd

# Create traces
traces = []
for index, row in df_recentlyPlayed.iterrows():
    start_time = row['datetime']
    end_time = start_time + pd.Timedelta(milliseconds=row['duration_ms'])
    y_value = row['tempo']
    traces.append(go.Scatter(
        x=[start_time, end_time],
        y=[y_value, y_value],
        mode='lines',
        fill='tozeroy',
        showlegend=False
    ))

# Create layout
layout = go.Layout(
    xaxis=dict(
        title='Time'
    ),
    yaxis=dict(
        title='Tempo'
    )
)

# Create figure
fig = go.Figure(data=traces, layout=layout)
fig.add_trace(go.Scatter(x=df_heartRate['datetime'], y=df_heartRate['bpm'], name='Heart Rate', marker=dict(color='blue')))

# Create Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1('Graph 1'),
    dcc.Graph(
        id='graph1',
        figure=fig
    )
])

# # Run app
# if __name__ == '__main__':
#     app.run_server(debug=True, port=8050)


In [ ]:
from fitbit import Fitbit
import os
from fitbit.exceptions import HTTPTooManyRequests

# from fitbit.rate_limits import RateLimitChecker
import sys
sys.path.append('../..')
print(os.getcwd())
from src import getSecret
# create a Fitbit client

client_id = getSecret('FITBIT_CLIENT_ID')
client_secret = getSecret('FITBIT_CLIENT_SECRET')
fitbit_client = Fitbit(client_id=client_id, client_secret=client_secret, access_token=access_token, refresh_token=refresh_token)

# create a RateLimitChecker using the client and the API endpoint you want to check
# rate_limit_checker = RateLimitChecker(fitbit_client, 'activities')

# # check the number of requests left
# requests_left = rate_limit_checker.requests_left()

# # print the number of requests left
# print(f"There are {requests_left} requests left for activities API endpoint.")

try:
    # make an API request
    data = fitbit_client.intraday_time_series('activities/heart', base_date='2021-04-08', detail_level='1sec')

except HTTPTooManyRequests as e:
    # get rate limit status
    print(e, e.retry_after_secs)
    attrs = vars(e)
    for item in attrs:
        print(item,":",attrs[item])
    # rate_limit_status = fitbit_client.client.rate_limit_status()

    # check remaining requests
    # remaining_requests = rate_limit_status['user']['remaining']

    # print(f"API rate limit exceeded. {remaining_requests} requests remaining.")


In [ ]:
fitbit_client.client.api_call_limit

In [ ]:
attrs = vars(fitbit_client.client.session)
# print(', '.join("%s: %s" % item for item in attrs.items()))
for key, value in attrs.items():
    print(key, value)

In [ ]:
{"acousticness":0.325,
 "analysis_url":"https://api.spotify.com/v1/audio-analysis/6UnfPP7ChYwQ2h3uTduRA6",
 "artist":"Ajja",
 "danceability":0.405,
 "duration_ms":295482,
 "energy":0.232,
 "id":"6UnfPP7ChYwQ2h3uTduRA6",
 "instrumentalness":0.849,
 "key":4,
 "liveness":0.111,
 "loudness":-15.973,
 "mode":0,
 "name":"Stage Left",
 "played_at":"Thu, 13 Apr 2023 01:06:46 GMT",
 "songImage":"https://i.scdn.co/image/baee872da6b7b0981d555ee7cff9e73845111e8e",
 "song_id":"6UnfPP7ChYwQ2h3uTduRA6",
 "speechiness":0.0304,
 "tempo":73.734,
 "time_signature":4,
 "track_href":"https://api.spotify.com/v1/tracks/6UnfPP7ChYwQ2h3uTduRA6",
 "type":"audio_features",
 "uri":"spotify:track:6UnfPP7ChYwQ2h3uTduRA6",
 "valence":0.118}

In [ ]:
display(df_heartRate.head(4))
display(df_recentlyPlayed.head(4))


In [ ]:
import plotly.graph_objs as go

traces = []
for index, row in df_recentlyPlayed.iterrows():
    start_time = row['datetime']
    end_time = start_time + pd.Timedelta(milliseconds=row['duration_ms'])
    y_value = row['tempo']
    traces.append(go.Scatter(
        x=[start_time, end_time],
        y=[y_value, y_value],
        mode='lines',
        fill='tozeroy',
        showlegend=False

    ))

layout = go.Layout(
    xaxis=dict(
        title='Time'
    ),
    yaxis=dict(
        title='Tempo'
    )
)

fig = go.Figure(data=traces, layout=layout)
fig.add_trace(go.Scatter(x=df_heartRate['datetime'], y=df_heartRate['bpm'], name='Heart Rate', marker=dict(color='blue')))

fig.show()
